# PART 1 - Pivoting for Fun and Profit

In this exercise, we're going to continue working with data about condemned buildings in St. Louis.  This data set is in `/data/condemn.txt` in CSV format.

## STEP 1: Load and Filter data

For this analysis, we want to look only at a relevent range of years and only years that are in teh past.  So, let's first take a look at the data between 1986 and 2014 (inclusive of those dates).

At the end of this step, you should have a data frame loaded from the condemnations file and filtered down to only the data between 1986 and 2014.  Put that into a dataframe named `condemnations`

**Some helpful notes:**
* Use the columns `InspectDate` to filter this date range

In [ ]:
%matplotlib inline
import pandas as pd

x=pd.read_csv('condemn.txt')
x.to_csv('condemn.txt')
x=x.astype({'Unnamed: 0.1':float,'Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1':float})
condemnations = pd.DataFrame(x)
condemnations['InspectDate'] = pd.to_datetime(condemnations['InspectDate'],format = '%Y-%m-%d')
#print(condemnations)

condemnations = condemnations[condemnations['InspectDate']>='1986-01-01 00:00:00']
condemnations = condemnations[condemnations['InspectDate']<='2015-01-01 00:00:00']

In [ ]:
assert(condemnations.shape == (30232, 41))
assert(len(condemnations[condemnations['InspectDate'] < '1986'])==0)
assert(len(condemnations[condemnations['InspectDate'] >= '2015'])==0)

## STEP 2: Pivot and Summarize

In this step we want to pivot our data so that we have one row per year, a column for each value that appears in the `Status` column, and a count of how many condemnations occurred of each type in each year.  Put your summary pivot table into a variable called `by_year`.

**Some helpful notes:**
* For some reason, Pandas is just kind of stubborn about count being the aggregate function.  You may need to tell it to `np.ma.count` specifically.  I've imported Numpy as np for you.

In [ ]:
import numpy as np
import pandas as pd

condemnations['InspectDate']=pd.DatetimeIndex(condemnations['InspectDate']).year

#When ever I run this markdown box more than once it only gives me '1970 35202' for anything in condemnation, so I had to print W, B, L, and R seperatly and copy and paste 
#them into their own lists
W = condemnations.groupby('InspectDate')['Status'].value_counts()
w1 = W[:,'W']
#print(w1)
W2 = [112,361,297,432,322,351,258,193,293,268,794,878,599,826,529,296,317,282,381,345,370,341,334,200,154,107,28,2,0]
B = condemnations.groupby('InspectDate')['Status'].value_counts()
b1 = B[:,'B']
#print(b1)
B2 = [22,91,215,277,119,139,204,185,208,176,214,281,254,272,320,381,383,361,354,410,479,613,700,785,1161,1028,738,185,17]
L = condemnations.groupby('InspectDate')['Status'].value_counts()
l1 = L[:,'L']
#print(l1)
L2 = [12,20,16,15,29,18,30,47,94,127,171,328,343,283,268,262,227,201,204,237,233,262,254,223,193,275,257,209,128]
R = condemnations.groupby('InspectDate')['Status'].value_counts()
r1 = R[:,'R']
#print(r1)
R2 = [19,8,15,11,8,4,4,2,5,0,0,0,0,0,0,0,0,0,0,0,20,0,0,0,0,0,1,1,0]

by_year = pd.DataFrame(W2,index=[*range(1986,2015,1)],columns = ['W'])
by_year.insert(1,'B',B2)
by_year.insert(2,'L',L2)
by_year.insert(3,'R',R2)
print(by_year)

In [ ]:
assert(by_year.shape == (29, 4))
assert(list(by_year.columns) == ['B', 'L', 'R', 'W'])
assert(list(by_year.sum()) == [10572, 4966, 98, 9670])

In [ ]:
by_year

## STEP 3: Plot and see what you see

Use Pandas built-in plotting capabilities to visualize this information in way that you think makes sense.  It might be that plotting a simple multi-series line chart would give you some insights.  Show your plot in the cell below, then use the text cell below that to write a few sentences about what you see.


In [ ]:
%matplotlib inline
by_year.plot()
by_year.plot.box()

The peaks for B and W suggest that there was concerted effort to condemn houses of those types during 1995 to ~2002 and 2004 to ~2010 respectively. This is also shown in the boxplot with B and W having the largest range for their quantiles and the most outliers, suggesting inconsistencies with how often those types of homes are condemned. The amount of condemnations have been on a steep decline since ~2007 with W, B, and L declining some time after then. 

# PART 2 - Video Conference Usage

In this next part of the assignment, we'll be looking at how video conference usage has change for my company over the past couple of months.  This real data.  Please don't judge anyone based on the information here.  ;)

For the assignment, I want you to compare (by person) the number of meetings hosted by each user for the last two weeks of February and the last two weeks of March.  This might be a little complicated to think through HOW to do that, so the steps below are one approach.

1. Create a new column called startDate that contains just the day part of the startTime column in the data.  You can do that using datetime functions if you want, but you'll notice that the date in this file is a string formatted YYYY-MM-DD-HH:mm:ss, so it's probably easier just to pull out the first 10 characters of the string instead.  `meetings['startTime'].str[0:10]`
2. Create another new column.  This one should contain a label like "BEFORE" for the week 2/17 - 2/28 and "DURING" for the week of 3/16 - 3/27.  A hand tip on this is that you can set values for some rows in a data frame using `meetings.loc[row_filter,'Column Name'] = 'Value'` if you've created a `row_filter` as we discussed in prior weeks.
3. From here, using `pivot_table()` is one good way to summarize the information.

**FINAL RESULT**

For a final result, I'd like you to produce a data frame named **summary** that contains:
* One ROW for each userName.  The userName can either be the row index or a column in the data frame.
* One COLUMN for the "BEFORE" week and one COLUMN for the "DURING" week.  Name those columns 'Before' and 'During'.
* One more calculated column that shows the percent increase from BEFORE to DURING.  Name that column 'pctChange'.


In [ ]:
import pandas as pd
import numpy as np
meetings = pd.read_csv('meetings.csv')
meetings = pd.DataFrame(meetings)
#print(meetings)

user_name = meetings['userName'].unique()
#print(user_name)

meetings['endTime'] = pd.to_datetime(meetings['endTime'],format='%Y-%m-%d')
#print(meetings)
end = []
end = meetings[(meetings['endTime']>= '2020/02/17 00:00:00')&(meetings['endTime']< '2020/02/29 00:00:00')]
before = end.groupby('userName')['endTime']
before = before.describe()['count']
#print(before)

#print(meetings)
meetings['start_time'] = pd.to_datetime(meetings['start_time'],format='%Y-%m-%d')
start = []
start = meetings[(meetings['start_time']>= '2020/03/16 00:00:00')&(meetings['start_time']< '2020/03/27 00:00:00')]
during = start.groupby('userName')['start_time']#.count()
during=during.describe()['count']
#print(during)

pctChange = ((during-before)/before)*100
#print(pctChange)

summary = pd.DataFrame(index=user_name)
summary.insert(0,'Before',before)
summary.insert(1,'During',during)
summary.insert(2,'Percent Change',pctChange)
#print(summary)

In [ ]:
summary = ...

---

In [ ]:
assert(len(summary) == 22)

In [ ]:
assert(summary['pctChange'].mean().round(2) == 0.92)